In [1]:
import os
import cv2
import numpy as np

# Example of loading and preprocessing images
def load_images(directory, image_size):
    images = []
    labels = []
    crop_types = os.listdir(directory)

    for crop_type in crop_types:
        path = os.path.join(directory, crop_type)
        for filename in os.listdir(path):
            img_path = os.path.join(path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)
            images.append(img)
            labels.append(crop_type)

    return np.array(images), np.array(labels)

# Example usage:
data_directory = 'plant1'
image_size = (56, 56)
images, labels = load_images(data_directory, image_size)


In [2]:
print(len(labels))

4972


In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

def extract_features(images):
    # Load pre-trained CNN model
    model = ResNet50(weights='imagenet', include_top=False, input_shape=image_size + (3,))

    # Preprocess images for the ResNet50 model
    preprocessed_images = preprocess_input(images)

    # Extract features using the pre-trained model
    features = model.predict(preprocessed_images)

    return features.reshape(features.shape[0], -1)

# Example usage:
image_features = extract_features(images)

156/156 [==============================] - 60s 366ms/step


In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_svm_model(X_train, y_train):
    svm_model = SVC(kernel='linear', C=1.0)
    svm_model.fit(X_train, y_train)
    return svm_model

# Train the model 
X_train, X_test, y_train, y_test = train_test_split(image_features, labels, test_size=0.2, random_state=42)
svm_model = train_svm_model(X_train, y_train)

In [5]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

# Example usage:
evaluate_model(svm_model, X_test, y_test)

Accuracy: 0.8864321608040201


In [6]:
import streamlit as st 

def choose_file():
    file = st.file_uploader("Please upload the plant image",type=["jpg","png"])
    return file

In [8]:
def predict_crop_type(model, image):
    image = cv2.resize(image, image_size)
    feature = extract_features(np.expand_dims(image, axis=0))
    crop_type = model.predict(feature)
    return crop_type[0]


# Example usage:
new_image_path  = "C:/Users/DELL/Documents/crop identification/plant1/Apple___Cedar_apple_rust/0cd24b0c-0a9d-483f-8734-5c08988e029f___FREC_C.Rust 3762_newGRR.JPG"
new_image = cv2.imread(new_image_path)
predicted_crop_type = predict_crop_type(svm_model, new_image)
print("Predicted Crop Type:", predicted_crop_type)


1/1 [==============================] - 2s 2s/step
Predicted Crop Type: Apple___Cedar_apple_rust


In [14]:
import pickle


filename = 'D:/machine_Learning_projects/plant_crop_identification_machine_learning/model/svm_crop_identifiaction.pkl'

# # Save model (serialize)
pickle.dump(svm_model, open(filename, 'wb'))

# # Load model (de-serialize)
# pickle.load(open(filename, 'rb'))
